In [ ]:
confirmed_csv = "corona_confirmed.csv"
confirmed_gitpath = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

In [ ]:
!curl -o $confirmed_csv $confirmed_gitpath 

In [ ]:
import pandas as pd
df = pd.read_csv(confirmed_csv)

df = df.melt(id_vars=["Province/State","Country/Region","Lat","Long"],
                var_name="DateColumns",
                value_name="CasesColumns")

df['DateColumns'] = pd.to_datetime(df['DateColumns'], format='%m/%d/%y').dt.strftime('%Y-%m-%d')
df["Province/State"] = df["Province/State"].fillna(df["Country/Region"])
df['DateColumns'] = pd.to_datetime(df['DateColumns'])
df["CasesColumns"] = df["CasesColumns"].astype(int)
df['Lat'].fillna(df['Lat'].mean(), inplace=True)
df['Long'].fillna(df['Long'].mean(), inplace=True)

In [ ]:
burning_counteries = df.groupby("Country/Region")["CasesColumns"].max().sort_values(ascending=False).reset_index()
burning_counteries_lts = [*burning_counteries["Country/Region"]][:20]
#ilk 20 nin içine türkiyeyi tekrardan almıyorum çünkü zaten şu an güncel olarak ilk 10 da olduğu için  
# ben tekrar alırsam 2 defa sayıyor ve kodun ileri aşamalarında hata vermesini sağlıyor
burning_counteries_lts

In [ ]:
df_burning_counteries = burning_counteries[burning_counteries["Country/Region"].isin(burning_counteries_lts)].reset_index(drop = "True")
df_burning_counteries["Country/Region"] = df_burning_counteries["Country/Region"].str.replace(",", "")
df_burning_counteries

In [ ]:
burning_counteries_daily = df.groupby(["DateColumns","Country/Region"])["CasesColumns"].max().sort_values(ascending=True).reset_index()
burning_counteries_daily = burning_counteries_daily[burning_counteries_daily["Country/Region"].isin(burning_counteries_lts)].reset_index(drop = "True")
burning_counteries_daily["Country/Region"] = burning_counteries_daily["Country/Region"].str.replace(",","")
burning_counteries_daily

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
nrow = 5
ncol = 4

df_burning_counteries_grouped = burning_counteries_daily.groupby("Country/Region")

df_burning_counteries_names = [x[0] for x in [*df_burning_counteries_grouped]]

countery_df_list_formatted = []
countery_df_list = [x[1].drop(columns=["Country/Region"]).reset_index(drop=True) for x in [*df_burning_counteries_grouped]]
for df_countery in countery_df_list:
    df_countery.set_index("DateColumns",inplace=True)
    df_countery.index = pd.to_datetime(df_countery.index)
    countery_df_list_formatted.append(df_countery.diff().dropna())
    
fig, axes = plt.subplots(nrow, ncol, figsize=(15,13), sharex=True)
plt.subplots_adjust(hspace=0.5)

myFmt =mdates.DateFormatter("%Y-%m-%d")
for axi in axes.flat:
    axi.xaxis.set_major_locator(plt.MaxNLocator(7))
    axi.xaxis.set_major_formatter(myFmt)
for r in range(nrow):
    for c in range(ncol):
        countery_df_list_formatted[((nrow-1)*r)+c].plot(ax=axes[r,c], title=df_burning_counteries_names[((nrow-1)*r)+c])